# AiiDA mini-demo
## Automatic calculations with pseudos from SSSP and k-points from SeeK-path

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import DataFactory, CalculationFactory, Code, load_node
ParameterData=DataFactory('parameter')
import os, time, numpy, pylab
from IPython.display import Image
%matplotlib inline

#### Load the importer from the COD database, and get a silicon supercell (take the first query result)

In [ ]:
from aiida.tools.dbimporters import DbImporterFactory
CodImporter = DbImporterFactory('cod')
importer = CodImporter()

In [ ]:
all_si_results = list(importer.query(spacegroup="F d -3 m :1", formula="Si"))
cod_result = all_si_results[0] # e.g.: ID 9008565

#### Convert it to an explicit AiiDA structure, show cell and coordinates

In [ ]:
structure = cod_result.get_aiida_structure()

In [ ]:
print structure._exportstring('xsf')

#### Get primitive cell from SeeK-path (also standardized according to crystallographic conventions), and show some information

In [ ]:
from seekpath.aiidawrappers import get_path, get_explicit_k_path
seekpath_info = get_path(structure)
primitive_structure = seekpath_info['primitive_structure']
explicit_path = get_explicit_k_path(structure)

In [ ]:
print "Bravais lattice: {}\nSpacegroup: {}\nInput cell had a volume {}x w.r.t. the primitive cell".format(
    seekpath_info['bravais_lattice_extended'], 
    seekpath_info['spacegroup_international'], 
    int(seekpath_info['volume_original_wrt_prim']))
print "Suggested path: ",
print ", ".join("{}-{}".format(p1, p2) for p1, p2 in seekpath_info['path'])
print "K-point coordinates (scaled units):"
for label, coords in seekpath_info['point_coords'].iteritems():
    print "{:7s} {:18.10f} {:18.10f} {:18.10f}".format(label, *coords)

In [ ]:
Image(filename='../common/data/bz-cF2.png')

#### Get suggested cutoff from SSSP

In [ ]:
SSSP_eff_cutoffs = {l.split()[0]: int(l.split()[1]) 
                    for l in open(os.path.join(
                        os.path.dirname(__name__),os.pardir,
                        'common','data','SSSP_acc_cutoffs.txt')).readlines() 
                    if l.split()[1] != '??'}

In [ ]:
for elem in sorted(SSSP_eff_cutoffs)[:10]:
    print "{:2s}: {:3.0f} Ry".format(elem, SSSP_eff_cutoffs[elem])
print "   ..."

#### Pick the suggested cutoff for this structure, as the largest cutoff for all species in the system

In [ ]:
SSSP_cutoff = float(max(SSSP_eff_cutoffs[sym] for sym in structure.get_symbols_set()))
print "Suggested cutoff: {} Ry".format(SSSP_cutoff)

#### Pick a suitable regular mesh from a given density; here: 0.4 Angstrom<sup>-1</sup>

In [ ]:
KpointsData = DataFactory('array.kpoints')
kpts = KpointsData()
kpts.set_cell_from_structure(primitive_structure)
kpts.set_kpoints_mesh_from_density(distance=0.4, offset=[0.,0.,0.])

In [ ]:
print kpts.get_kpoints_mesh()

#### Generate a Quantum ESPRESSO relaxation calculation and submit it

In [ ]:
calc = Code.get_from_string('pw-5.1@localhost').new_calc()
calc.set_resources({'num_machines': 1})

In [ ]:
calc.use_structure(primitive_structure)
calc.use_kpoints(kpts)
calc.use_pseudos_from_family("SSSP")

In [ ]:
calc.use_parameters(ParameterData(dict={
    'CONTROL': {
        'calculation': 'relax',
        'restart_mode': 'from_scratch',
    },
    'SYSTEM': {
        'ecutwfc': SSSP_cutoff,
        'ecutrho': 8.*SSSP_cutoff,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }}))

In [ ]:
calc.store_all()
calc.submit()

In [ ]:
while calc.get_state() != 'FINISHED':
    print ".",
    time.sleep(5)
    

In [ ]:
#calc = load_node(5049)

#### Perform the bands calculation 

In [ ]:
bandscalc = calc.create_restart(use_output_structure=True)
bandscalc.set_resources({'num_machines': 1})
bandscalc.use_kpoints(explicit_path['explicit_kpoints'])

bands_parameters = calc.inp.parameters.get_dict()
bands_parameters['CONTROL']['calculation'] = 'bands'
bandscalc.use_parameters(ParameterData(dict=bands_parDoameters))

bandscalc.use_settings(ParameterData(dict={
    'also_bands': True
}))

In [ ]:
bandscalc.store_all()
bandscalc.submit()

In [ ]:
while bandscalc.get_state() != 'FINISHED':
    print ".",
    time.sleep(5)

In [ ]:
# bandscalc = load_node(5059)

#### Prepare the data to be plotted

In [ ]:
import json
plot_data = json.loads(
        "\n".join([_.partition('#')[0] 
                   for _ in bandscalc.out.output_band._exportstring('json').splitlines() 
                   if _.partition('#')[0].split()]))

#### Configuring pylab and doing the final plotting

In [ ]:
curpos = 0
last_label = None
f = pylab.subplot(1,1,1)
pylab.axvline(curpos)
ticks = []
tick_labels = []
for path in plot_data['paths']:
    if last_label is None:
        ticks.append(curpos)
        tick_labels.append(path['from'])
        last_label = path['from']
    if last_label != path['from']:
        tick_labels[-1] += "|{}".format(path['from'])
    if path['length'] == 1:
        continue
    newlen = 0
    for band in path['values']:
        newlen = len(band)
        pylab.plot(range(curpos, curpos+newlen), band, 'k')
    curpos += newlen
    if newlen == 2:
        print path
    pylab.axvline(curpos) 
    ticks.append(curpos)
    tick_labels.append(path['to'])
    last_label = path['to']
    
pylab.xlim([0, curpos])
f.set_xticks(ticks)

tick_labels = ["$\Gamma$" if _ == "GAMMA" else _ for _ in tick_labels ]

f.set_xticklabels(tick_labels)
print 'Band structure:'